In [41]:
import duckdb
import pandas as pd
from statsbombpy import sb
import polars as pl
import time

# DuckDB bağlantısı
#con = duckdb.connect("bitirme_test.db")


In [42]:
def get_homeTeam(data):
    team = data["home_team"].unique()
    return team

In [43]:
def pass_data_creation(events_extra_info,match_ID):
    
    events_df = sb.events(match_id=match_ID)
    events_df = events_df.set_index("match_id").join(
        events_extra_info.set_index("match_id"), how="left", rsuffix='_match'
    ).reset_index()

    # Koordinatları ayır
    events_df[['x', 'y']] = events_df['location'].apply(pd.Series)
    events_df[['pass_end_x', 'pass_end_y']] = events_df['pass_end_location'].apply(pd.Series)

    # Paslar tablosunu oluştur
    passes = events_df[(events_df.type == "Pass")]
    passes = passes[passes["pass_outcome"] != "Injury Clearance"]
    shots = events_df[(events_df.type == "Shot")][["player","shot_type","shot_key_pass_id","shot_outcome","shot_statsbomb_xg"]].dropna()
    passes_M = passes.set_index("id").join(
        shots.set_index("shot_key_pass_id"), how="left", rsuffix='_shotFromPass'
    )
    passes_M = passes_M[passes_M["pass_outcome"] != "Injury Clearance"]

    passes_M["pass_succes"] = passes_M.pass_outcome.isna()
    passes_M["f3rd_pass"] = (passes_M.x < 80) & (passes_M.pass_end_x > 80)
    passes_M["pass_progression"] = passes_M["pass_end_x"] - passes_M["x"]
    passes_M.loc[passes_M.pass_outcome.notna(), "pass_progression"] = None
    passes_M["shot_assisted_pass"] = (passes_M["pass_succes"] == True) &  (passes_M["shot_outcome_shotFromPass"].notna())
    passes_M["goal_assisted_pass"] = (passes_M["pass_succes"] == True) & (passes_M["shot_outcome_shotFromPass"] == "Goal")
    
    passes_M["cross_pass"] = (passes_M.pass_cross == True)
    passes_M["box_pass"] = (
        ((passes_M.x < 102) | ((passes_M.y < 18) | (passes_M.y > 62))) & ((passes_M.pass_end_x > 102) & ((passes_M.pass_end_y > 18) & (passes_M.pass_end_y < 62)))
    )

    passes_M["rowID"] = passes_M.index

    return passes_M

In [57]:
def foul_data_creation(events_extra_info,match_ID):

    events_df = sb.events(match_id=match_ID)
    events_df = events_df.set_index("match_id").join(
        events_extra_info.set_index("match_id"), how="left", rsuffix='_match'
    ).reset_index()

    # Koordinatları ayır
    events_df[['x', 'y']] = events_df['location'].apply(pd.Series)
    events_df[['pass_end_x', 'pass_end_y']] = events_df['pass_end_location'].apply(pd.Series)

    # Fauller tablosunu oluştur

    # Set of existing columns in the DataFrame
    existing_columns = set(events_df.columns)

    # Set of columns you want to check
    columns_to_check = {"foul_committed_advantage", "foul_committed_card"}

    # Find which columns exist and which do not
    missing = columns_to_check - existing_columns
    if len(missing) == 0:
        foul_commited = events_df[(events_df.type == "Foul Committed")][
        ["id", "type", "player", "position","team", "related_events", "foul_committed_advantage", "foul_committed_card", "x", "y"]
        ].set_index("id")

    elif len(missing) > 1:
        print("there was no commited advantage nor commited card in this match ")
        foul_commited = events_df[(events_df.type == "Foul Committed")][
            ["id", "type", "player","position","team", "related_events", "x", "y"]]
        foul_commited["foul_committed_advantage_foul_commitedJ"] = None
        foul_commited["foul_committed_card_foul_commitedJ"] = None
        foul_commited = foul_commited.set_index("id")
    
    elif "foul_committed_card" in missing:
        print("there was no commited card in this match ")
        foul_commited = events_df[(events_df.type == "Foul Committed")][
            ["id", "type", "player","position","team", "related_events", "foul_committed_advantage", "x", "y"]]
        foul_commited["foul_committed_card_foul_commitedJ"] = None
        foul_commited = foul_commited.set_index("id")
    
    elif "foul_committed_advantage" in missing:
        print("there was no commited advantage in this match ")
        foul_commited = events_df[(events_df.type == "Foul Committed")][
            ["id", "type", "player","position","team", "related_events", "foul_committed_card", "x", "y"]]
        foul_commited["foul_committed_advantage_foul_commitedJ"] = None
        foul_commited = foul_commited.set_index("id")

    
    fouls = events_df[(events_df.type == "Foul Won")].explode("related_events")
    fouls_M = fouls.set_index("related_events").join(foul_commited, how="left", rsuffix='_foul_commitedJ')
    fouls_M = fouls_M[fouls_M["player_foul_commitedJ"].notna()]
    fouls_M["rowID"] = fouls_M.index
    return fouls_M

In [45]:
matches = sb.matches(competition_id=11, season_id=27)
events_extra_info = matches[["match_id","season","competition","home_team","away_team","referee"]]


/Users/emirysaglam/GitHub/Bitirme/.venv/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [46]:
info = events_extra_info[events_extra_info["match_id"]==3825838]
info["competition"]

28    Spain - La Liga
Name: competition, dtype: object

In [39]:

# Start timer
start_time = time.perf_counter()

# Code to be timed
# ...
passDF = pass_data_creation(events_extra_info,3825838)

# End timer
end_time = time.perf_counter()

# Calculate elapsed time
print(end_time - start_time)

/Users/emirysaglam/GitHub/Bitirme/.venv/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


2.844970792008098


In [29]:
passDF[["match_id","season","competition","home_team","away_team","referee",
    "player",
    "x",
    "y",
    "pass_end_x",
    "pass_end_y",
    "player_shotFromPass",
    "shot_outcome_shotFromPass",
    "shot_statsbomb_xg_shotFromPass",
    "pass_succes",
    "f3rd_pass",
    "pass_progression",
    "shot_assisted_pass",
    "goal_assisted_pass",
    "cross_pass",
    "pass_cross",
    "box_pass"
]].head(10)

,match_id,season,competition,home_team,away_team,referee,player,x,y,pass_end_x,...,shot_outcome_shotFromPass,shot_statsbomb_xg_shotFromPass,pass_succes,f3rd_pass,pass_progression,shot_assisted_pass,goal_assisted_pass,cross_pass,pass_cross,box_pass
id,,,,,,,,,,,,,,,,,,,,,
972873a7-3d00-4f4a-8d9a-cc429f3eb973,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Saúl Ñíguez Esclapez,61.0,40.1,60.3,...,NaN,NaN,True,False,-0.7,False,False,False,NaN,False
e4c05258-39a5-4c9c-97f7-6ebc0b5b9d85,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Thomas Teye Partey,60.6,42.9,45.4,...,NaN,NaN,True,False,-15.2,False,False,False,NaN,False
92a1865e-d479-4c03-937e-f88b60123cd8,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Jorge Resurrección Merodio,46.2,43.1,45.0,...,NaN,NaN,True,False,-1.2,False,False,False,NaN,False
768060eb-6a53-4c72-aaba-8ef65cae7ac9,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Filipe Luís Kasmirski,45.9,14.9,92.5,...,NaN,NaN,False,True,NaN,False,False,False,NaN,False
f148b397-1564-4331-83ac-d173330a0953,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Gorka Iraizoz Moreno,28.4,51.3,41.5,...,NaN,NaN,True,False,13.1,False,False,False,NaN,False
6bf4307b-6c1d-467e-b770-bb75f2e6d83a,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Xabier Etxeita Gorritxategi,44.0,20.2,39.5,...,NaN,NaN,True,False,-4.5,False,False,False,NaN,False
7c84e0c2-177b-444f-848e-1e1e3558be02,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Eneko Bóveda Altube,39.3,53.1,91.1,...,NaN,NaN,False,True,NaN,False,False,False,NaN,False
4bf6bb6d-633d-48b5-b225-6b6b36041f6c,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Filipe Luís Kasmirski,29.0,11.0,42.6,...,NaN,NaN,True,False,13.6,False,False,False,NaN,False
ebbe304c-039d-4f3b-9086-27c10074729b,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Saúl Ñíguez Esclapez,42.2,15.7,28.4,...,NaN,NaN,True,False,-13.8,False,False,False,NaN,False


In [40]:
passDF[[
    "match_id",
    "competition",
    "home_team",
    "away_team",
    "referee",
    "player",
    "x",
    "y",
    "pass_end_x",
    "pass_end_y",
    "player_shotFromPass",
    "shot_outcome_shotFromPass",
    "shot_statsbomb_xg_shotFromPass",
    "pass_succes",
    "f3rd_pass",
    "pass_progression",
    "shot_assisted_pass",
    "goal_assisted_pass",
    "box_pass"
]][passDF["shot_outcome_shotFromPass"]=="Goal"]

,match_id,competition,home_team,away_team,referee,player,x,y,pass_end_x,pass_end_y,player_shotFromPass,shot_outcome_shotFromPass,shot_statsbomb_xg_shotFromPass,pass_succes,f3rd_pass,pass_progression,shot_assisted_pass,goal_assisted_pass,box_pass
id,,,,,,,,,,,,,,,,,,,
9ea92325-9eaf-420d-b2a4-a1e41b1010d4,3825838,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Antoine Griezmann,91.3,58.1,107.6,39.0,Fernando José Torres Sanz,Goal,0.098868,True,False,16.3,True,True,True


In [62]:
print(get_homeTeam(passDF))


['Athletic Club']


In [58]:
foulsDF = foul_data_creation(events_extra_info,3825838)

/Users/emirysaglam/GitHub/Bitirme/.venv/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


there was no commited card in this match 


In [60]:
for i in foulsDF.columns:
    print(i)

match_id
50_50
ball_receipt_outcome
ball_recovery_offensive
ball_recovery_recovery_failure
block_deflection
block_offensive
carry_end_location
clearance_aerial_won
clearance_body_part
clearance_head
clearance_left_foot
clearance_right_foot
counterpress
dribble_nutmeg
dribble_outcome
dribble_overrun
duel_outcome
duel_type
duration
foul_committed_advantage
foul_committed_type
foul_won_advantage
foul_won_defensive
goalkeeper_body_part
goalkeeper_end_location
goalkeeper_outcome
goalkeeper_position
goalkeeper_technique
goalkeeper_type
id
index
injury_stoppage_in_chain
interception_outcome
location
minute
miscontrol_aerial_won
off_camera
out
pass_aerial_won
pass_angle
pass_assisted_shot_id
pass_body_part
pass_cross
pass_cut_back
pass_deflected
pass_end_location
pass_goal_assist
pass_height
pass_inswinging
pass_length
pass_no_touch
pass_outcome
pass_outswinging
pass_recipient
pass_recipient_id
pass_shot_assist
pass_switch
pass_technique
pass_through_ball
pass_type
period
play_pattern
player
p

In [65]:
foulsDF["foul_committed_advantage_foul_commitedJ"].unique()

array([None], dtype=object)

In [61]:
columns = [
    "competition",
    "home_team",
    "away_team",
    "referee",
    "x",
    "y",
    "player",
    "position",
    "possession_team",
    "type_foul_commitedJ",
    "player_foul_commitedJ",
    "position_foul_commitedJ",
    "team_foul_commitedJ",
    "related_events",
    "foul_committed_advantage_foul_commitedJ",
    "x_foul_commitedJ",
    "y_foul_commitedJ",
    "foul_committed_card_foul_commitedJ",
    "rowID"
]


foulsDF[columns]

,competition,home_team,away_team,referee,x,y,player,position,possession_team,type_foul_commitedJ,player_foul_commitedJ,position_foul_commitedJ,team_foul_commitedJ,related_events,foul_committed_advantage_foul_commitedJ,x_foul_commitedJ,y_foul_commitedJ,foul_committed_card_foul_commitedJ,rowID
related_events,,,,,,,,,,,,,,,,,,,
792595c3-e8fc-48ec-83f9-0b38175cde94,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,50.1,18.4,Saúl Ñíguez Esclapez,Left Midfield,Athletic Club,Foul Committed,Raúl García Escudero,Center Attacking Midfield,Athletic Club,[4b5794a0-8631-4928-886b-4b1718fda5a1],NaN,70.0,61.7,None,792595c3-e8fc-48ec-83f9-0b38175cde94
8c64e7e5-a6a8-4e28-93ca-079fd67d90fb,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,20.4,70.8,Óscar de Marcos Arana,Right Back,Atlético Madrid,Foul Committed,Saúl Ñíguez Esclapez,Left Midfield,Atlético Madrid,[7ff0e14e-3c70-490c-9b13-08e48008ab71],NaN,99.7,9.3,None,8c64e7e5-a6a8-4e28-93ca-079fd67d90fb
32dacfef-4fb6-4d62-be24-7bf383f5a55c,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,79.6,16.4,Raúl García Escudero,Center Attacking Midfield,Athletic Club,Foul Committed,Augusto Matías Fernández,Right Defensive Midfield,Atlético Madrid,[7bb46b3e-e39c-456e-aedc-0d13fa8f9013],True,40.5,63.7,None,32dacfef-4fb6-4d62-be24-7bf383f5a55c
69d9c8ae-0040-491b-92b7-b0790f0e4dd4,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,63.6,21.3,Jorge Resurrección Merodio,Left Defensive Midfield,Atlético Madrid,Foul Committed,Mikel Balenziaga Oruesagasti,Left Back,Athletic Club,"[6e8830a1-ce0e-4a81-901d-2715b618f624, a8f0db8...",True,56.5,58.8,None,69d9c8ae-0040-491b-92b7-b0790f0e4dd4
8d23fdbf-8989-4265-895e-633f920a2379,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,94.5,23.3,Iker Muniain Goñi,Left Wing,Athletic Club,Foul Committed,Thomas Teye Partey,Right Midfield,Atlético Madrid,[403fb547-3150-4b3d-b9dc-9f0401630792],NaN,25.6,56.8,None,8d23fdbf-8989-4265-895e-633f920a2379
d16991bd-408e-4dce-a364-fd8e0716e9c2,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,4.3,44.1,Óscar de Marcos Arana,Right Back,Atlético Madrid,Foul Committed,Thomas Teye Partey,Right Midfield,Atlético Madrid,[8f48fc0e-f67a-4b7e-a518-404ce6571734],NaN,115.8,36.0,None,d16991bd-408e-4dce-a364-fd8e0716e9c2
8210310c-de58-4bde-8f6c-20f661097cb5,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,68.3,41.4,Antoine Griezmann,Left Center Forward,Atlético Madrid,Foul Committed,Beñat Etxebarria Urkiaga,Left Defensive Midfield,Athletic Club,"[1bc610ec-3ca2-4f98-8bc8-039ba605145f, 4f43502...",True,51.8,38.7,None,8210310c-de58-4bde-8f6c-20f661097cb5
4dc204ce-4a9c-4e9f-8c91-115bddf68950,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,42.1,68.5,Markel Susaeta Laskurain,Right Wing,Athletic Club,Foul Committed,Saúl Ñíguez Esclapez,Left Midfield,Atlético Madrid,[a76be9e0-b274-421b-b711-d3dbcbfe97b8],NaN,78.0,11.6,None,4dc204ce-4a9c-4e9f-8c91-115bddf68950
946fbe70-e401-4f37-8b66-e10a17d57ae4,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,3.8,9.0,Lucas Hernández Pi,Left Center Back,Athletic Club,Foul Committed,Markel Susaeta Laskurain,Right Wing,Athletic Club,[a4d26cba-558f-4745-ad8b-9b63c74e3a1a],NaN,116.3,71.1,None,946fbe70-e401-4f37-8b66-e10a17d57ae4


In [66]:

#con.register("passes",passes_M)
#con.register("fouls",fouls_M)

In [67]:
#con.table("passes").show()


In [ ]:
passDF = pd.DataFrame()
foulsDF= pd.DataFrame()

matches = sb.matches(competition_id=11, season_id=27)
events_extra_info = matches[["match_id","season","competition","home_team","away_team","referee"]]


for match_ID in [3825848,3825895]:

    #passData = pass_data_creation(events_extra_info,match_ID)

    foulData = foul_data_creation(events_extra_info,match_ID)

    #passDF = pd.concat([passDF, passData], axis=0, ignore_index=True)
    foulsDF = pd.concat([foulsDF, ], axis=0, ignore_index=True)




/Users/emirysaglam/GitHub/Bitirme/bitirme-venv/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/emirysaglam/GitHub/Bitirme/bitirme-venv/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/emirysaglam/GitHub/Bitirme/bitirme-venv/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/emirysaglam/GitHub/Bitirme/bitirme-venv/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/emirysaglam/GitHub/Bitirme/bitirme-venv/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [69]:
passDF[["match_id","season","competition","home_team","away_team","referee",
    "player",
    "x",
    "y",
    "pass_end_x",
    "pass_end_y",
    "player_shotFromPass",
    "shot_outcome_shotFromPass",
    "shot_statsbomb_xg_shotFromPass",
    "pass_succes",
    "f3rd_pass",
    "pass_progression",
    "shot_assisted_pass",
    "goal_assisted_pass",
    "cross_pass",
    "box_pass"
]].tail(10)

,match_id,season,competition,home_team,away_team,referee,player,x,y,pass_end_x,...,player_shotFromPass,shot_outcome_shotFromPass,shot_statsbomb_xg_shotFromPass,pass_succes,f3rd_pass,pass_progression,shot_assisted_pass,goal_assisted_pass,cross_pass,box_pass
1795,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,David Simón Rodríguez Santana,29.0,75.1,46.9,...,NaN,NaN,NaN,True,False,-17.9,False,False,False,False
1796,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Antolín Alcáraz Viveros,66.6,77.2,61.0,...,NaN,NaN,NaN,True,False,5.6,False,False,False,False
1797,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Marco Andreolli,17.7,51.6,35.2,...,NaN,NaN,NaN,True,False,-17.5,False,False,False,False
1798,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Víctor Machín Pérez,60.3,33.5,72.6,...,NaN,NaN,NaN,True,False,-12.3,False,False,False,False
1799,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Kevin Gameiro,82.1,39.3,94.0,...,NaN,NaN,NaN,True,False,-11.9,False,False,False,False
1800,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Javier Varas Herrera,6.0,56.9,68.8,...,NaN,NaN,NaN,False,False,NaN,False,False,False,False
1801,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Timothée Kolodziejczak,51.3,24.7,57.0,...,NaN,NaN,NaN,False,False,NaN,False,False,False,False
1802,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Vicente Iborra De La Fuente,49.8,50.4,37.5,...,NaN,NaN,NaN,True,False,12.3,False,False,False,False
1803,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Timothée Kolodziejczak,37.9,35.4,50.4,...,NaN,NaN,NaN,True,False,-12.5,False,False,False,False
1804,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Benoît Trémoulinas,53.5,10.5,77.4,...,NaN,NaN,NaN,True,False,-23.9,False,False,False,False


In [148]:
passDF["away_team"].unique()

array(['Celta Vigo', 'Eibar', 'Sevilla'], dtype=object)